In [1]:
from whisper1 import WhisperTranscriptorAPI

/home/ali/hubert_transcriptor/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
whisper_transcriptor=WhisperTranscriptorAPI(model_path='openai/whisper-base.en',file_processing=True)
result = whisper_transcriptor.generate_timestamps(audio_path='/home/ali/Desktop/idrak_work/transcriptor_module-transcriptor-module/WTranscriptor/audios/backy.wav')

100%|██████████| 10245/10245 [00:27<00:00, 379.16frames/s]


In [3]:
transcript = result['text']

In [4]:
segments=result['segments']

In [11]:
def review(index):
    print(segments[index])
    print(segments[index]['text'],segments[index]['whole_word_timestamps'])

In [12]:
review(3)

{'id': 3, 'seek': 0, 'offset': 0.0, 'start': 6.6000000000000005, 'end': 9.200000000000001, 'text': " Yeah, that's me. Like what happened? What happened?", 'tokens': [50389, 18435, 13, 50419, 50513, 9425, 13, 18435, 13, 50563, 50563, 2011, 1438, 318, 47882, 422, 14017, 30582, 13, 1374, 389, 345, 1909, 30, 50693, 50693, 9425, 11, 326, 338, 502, 13, 4525, 644, 3022, 30, 1867, 3022, 30, 50823, 51008, 9425, 13, 51058, 51058, 770, 373, 546, 257, 649, 1181, 17153, 2457, 13, 51173, 51173, 1867, 389, 345, 2045, 379, 30, 51223, 51383, 16805, 13, 51408, 51443, 314, 1101, 7926, 13, 51473, 51473, 1867, 373, 326, 30, 51503, 51503, 16805, 13, 51523, 51523, 9425, 13, 51543, 51543, 314, 1244, 307, 4609, 11, 475, 345, 760, 11, 314, 1101, 8179, 257, 1310, 1643, 826, 783, 13, 51713, 51728], 'temperature': 0.0, 'avg_logprob': -0.4843400377746022, 'compression_ratio': 1.5215311004784688, 'no_speech_prob': 0.34248536825180054, 'alt_start_timestamps': [6.599999904632568, 6.639999866485596, 6.619999885559082, 

In [147]:
def check_next_segments(query_word,segments):
    for segment in segments:
        for words in segment['whole_word_timestamps']:
            if query_word in words['word']:
                return words['timestamp']+0.3
    return None


In [148]:
def get_word_index(query_word,words,segments):
    for word in words:
        if query_word in word['word']:
            print(query_word)
            return (word['timestamp'])
    return check_next_segments(query_word=query_word,segments=segments)

In [149]:
def search_for_words(segment,annotator_text,segments=''):
    annotator_text_words = annotator_text.split()
    word_segments = segment['whole_word_timestamps']
    start = get_word_index(annotator_text_words[0],word_segments,segments)
    end = get_word_index(annotator_text_words[-1],word_segments,segments)
    return start,end

In [150]:
def search_for_text_segment(whisper_response, annotator_text, sampling_rate=16000):
    """Searches for a given text segment within a Whisper response.

    Args:
        whisper_response (dict): A dictionary containing the Whisper response.
        annotator_text (str): The text to search for.
        sampling_rate (int): The sampling rate of the audio.

    Returns:
        A tuple containing the start and end time (in samples) of the text segment, as well as the segment dictionary.

    """
    # Get the segments from the Whisper response
    segments = whisper_response.get('segments', [])
    start,end = 0, 0
    # Search for the text segment within the Whisper response
    for i,segment in enumerate(segments):
        if annotator_text[:int(len(annotator_text)/3)] in segment['text']:
            # Compute the start and end times in samples and return them
            start,end = search_for_words(segment=segment,annotator_text=annotator_text,segments=segments[i+1:])
            if not start:
                start = segment['start']
            if not end:
                end = segment['end']
            start = int(start * sampling_rate)
            end = int(end * sampling_rate)
            return (start, end)

    # If the text segment wasn't found, return None
    return None

In [151]:
partial_transcript = "Yeah, go ahead, honey. Go ahead. I'm just "

In [152]:
result1=search_for_text_segment(whisper_response=result,annotator_text=partial_transcript)

Yeah,


In [153]:
result1,(result1[1]-result1[0])/16000

((664000, 694879), 1.9299375)

In [154]:
import soundfile as sf

data, samplerate = sf.read('/home/ali/Desktop/idrak_work/transcriptor_module-transcriptor-module/WTranscriptor/audios/backy.wav')

In [155]:
extracted_data=data[result1[0]:result1[1]]

In [156]:
sf.write('test_1.wav', extracted_data, samplerate)

In [146]:
transcript

" Hello. Yeah. Hello. My name is Becky from Senior Benefits. How are you today? Yeah, that's me. Like what happened? What happened? Yeah. This was about a new state regulated final. What are you looking at? Okay. I'm sorry. What was that? Okay. Yeah. I might be interested, but you know, I'm busy a little bit right now. I can talk while I shop, you know, like it's a shopping for me every day. That is a saying. The day without shopping, it's a waste of day. So, ha, yeah. So I'm shopping right now. Mm-hmm. Yeah, go ahead, honey. Go ahead. I'm just going to confirm a few pieces of information. Okay. What is that? It will only take a minute. Okay. I remember it was something like a deal like that, but like what is happening? Getting spread into me. Actually, hold on a second. Excuse me, like I would love to have three bars, three pairs of those I y'all. Would you like to learn more about it? I think it's going to match my high health. Yeah. Hello? I'm sorry. It's just, it's like I have a re

In [ ]:
def check_next_segments(query_word, segments):
    """Check for the next segments that contain the query word.

    Args:
        query_word (str): The query word to search for.
        segments (list): A list of segments to search through.

    Returns:
        The timestamp of the next segment containing the query word if found, otherwise None.
    """
    for segment in segments:
        for words in segment['whole_word_timestamps']:
            if query_word in words['word']:
                return words['timestamp'] + 0.3
    return None


def get_word_index(query_word, words, segments):
    """Get the timestamp of the word in the segment.

    Args:
        query_word (str): The query word to search for.
        words (list): A list of words to search through.
        segments (list): A list of segments to search through.

    Returns:
        The timestamp of the word in the segment if found, otherwise the timestamp of the next segment containing the word.
    """
    for word in words:
        if query_word in word['word']:
            print(query_word)
            return word['timestamp']
    return check_next_segments(query_word=query_word, segments=segments)


def search_for_words(segment, annotator_text, segments=''):
    """Search for the start and end times of a given text segment.

    Args:
        segment (dict): A dictionary containing the segment.
        annotator_text (str): The text to search for.
        segments (list): A list of segments to search through.

    Returns:
        A tuple containing the start and end times of the text segment.
    """
    annotator_text_words = annotator_text.split()
    word_segments = segment['whole_word_timestamps']
    start = get_word_index(annotator_text_words[0], word_segments, segments)
    end = get_word_index(annotator_text_words[-1], word_segments, segments)
    return start, end


def search_for_text_segment(whisper_response, annotator_text, sampling_rate=16000):
    """Search for a given text segment within a Whisper response.

    Args:
        whisper_response (dict): A dictionary containing the Whisper response.
        annotator_text (str): The text to search for.
        sampling_rate (int): The sampling rate of the audio.

    Returns:
        A tuple containing the start and end time (in samples) of the text segment, as well as the segment dictionary.
    """
    # Get the segments from the Whisper response
    segments = whisper_response.get('segments', [])
    start, end = 0, 0
    # Search for the text segment within the Whisper response
    for i, segment in enumerate(segments):
        if annotator_text[:int(len(annotator_text) / 3)] in segment['text']:
            # Compute the start and end times in samples and return them
            start, end = search_for_words(segment=segment, annotator_text=annotator_text, segments=segments[i + 1:])
            if not start:
                start = segment['start']
            if not end:
                end = segment['end']
            start = int(start * sampling_rate)
            end = int(end * sampling_rate)
            return start, end

    # If the text segment wasn't found, return None
    return None
